In [86]:
import zmq
import time
import socket
import os
import sys

# Check wich computer to decide where the things are mounted
comp_name=socket.gethostname()
print 'Computer: ' + comp_name

if 'passaro' in comp_name:
    repos_folder = os.path.abspath('/mnt/cube/earneodo/repos')
    experiment_folder = os.path.join('/mnt/cube/earneodo/bci_zf/')

sys.path.append(os.path.join(repos_folder, 'soundflow', 'sound_tools'))
sys.path.append(os.path.join(repos_folder, 'ephysflow'))

from file_tools import experiment as et
from rig_tools import open_ephys as oe, beagle_bone as bb

Computer: passaro


In [2]:
port = "5558"
context = zmq.Context()
print "Connecting to server..."
socket = context.socket(zmq.REQ)
socket.connect("tcp://127.0.0.1:%s" % port)

Connecting to server...


In [74]:
class OpenEphysEvents():
    def __init__(self, port='5558', ip='127.0.0.1'):
        self.ip = ip
        self.port = port
        self.socket = None
        self.context = None
        self.timeout = 1.
        self.last_cmd = None
        self.last_rcv = None

    
    def connect(self):
        url = "tcp://%s:%d" % (self.ip, int(self.port))
        self.context = zmq.Context()
        self.socket = self.context.socket(zmq.REQ)
        self.socket.RCVTIMEO = int(self.timeout*1000)
        self.socket.connect(url)

    def start_acq(self, ):
        if self.query_status('Acquiring'):
            print 'Already acquiring'
        else:
            self.send_command('StartAcquisition')
            if self.query_status('Acquiring'):
                print 'Acquisition Started'
            else:
                print 'Something went wrong starting acquisition'
    
    def stop_acq(self, ):
        if self.query_status('Recording'):
            print 'Cant stop acquistion while recording'
            
        elif not self.query_status('Acquiring'):
            print 'No acquisition running'
        
        else:
            self.send_command('StopAcquisition')
            if not self.query_status('Acquiring'):
                print 'Acquistion stopped'
            else:
                print 'Something went wrong stopping acquisition'

    def start_rec(self, rec_par={'CreateNewDir':'0', 'RecDir':None, 'PrependText':None, 'AppendText':None}):        
        ok_to_start = False
        
        if self.query_status('Recording'):
            print 'Already Recording'
        
        elif not self.query_status('Acquiring'):
            print 'Was not Acquiring'
            self.start_acq()
            if self.query_status('Acquiring'):
                ok_to_start = True
                print 'OK to start'
        else:
            ok_to_start = True
            print 'OK to start'
        
        if ok_to_start:
            rec_opt = ['{0}={1}'.format(key, value) 
                       for key, value in rec_par.iteritems() 
                       if value is not None]
            self.send_command(' '.join(['StartRecord'] + rec_opt))
            if self.query_status('Recording'):
                print 'Recording path: {}'.format(self.get_rec_path)
            else:
                print 'Something went wrong starting recording'
        else:
            'Did not start recording'
                              
    def stop_rec(self, ):
        if self.query_status('Recording'):
            self.send_command('StopRecord')
            if not self.query_status('Recording'):
                print 'Recording stopped'
            else:
                print 'Something went wrong stopping recording'
        else:
            print 'Was not recording'
            
    def get_rec_path(self):
        return self.send_command('GetRecordingPath')
    
    def query_status(self, status_query='Recording'):
        query_dict={'Recording': 'isRecording',
                   'Acquiring': 'isAcquiring'}
        
        status_queried = self.send_command(query_dict[status_query])
        return True if status_queried=='1' else False if status_queried=='0' else None
    
    def send_command(self, cmd):
        self.socket.send(cmd)
        self.last_cmd = cmd
        self.last_rcv = self.socket.recv()
        return self.last_rcv
    
    def close(self):
        self.stop_rec()
        self.stop_acq()
        self.context.destroy()
        

In [78]:
oe.close()
oe = OpenEphysEvents()
oe.connect()
oe.start_acq()

Already acquiring


In [81]:
oe.start_rec({'CreateNewDir': '1', 'PrependText': 'tuvieja'})

Was not Acquiring
Acquisition Started
OK to start
Recording path: <bound method OpenEphysEvents.get_rec_path of <__main__.OpenEphysEvents instance at 0x7f312c119998>>


In [83]:
oe.stop_rec()

Recording stopped


In [104]:
import os
sess_path = os.path.abspath('/usr/local/experiment/raw_data/testing')
rec_par = {'CreateNewDir': '1',
           'RecDir': None,
           'PrependText': None, 
           'Append_text': None}

txt = ['{0}={1}'.format(key, value) 
       for key, value in rec_par.iteritems() 
       if value is not None]

tuvieja = ' '.join(['StartRecord'] + txt)

In [84]:
tuvieja = ' '.join(['StartRecord'] + txt)
print tuvieja

StartRecord CreateNewDir=1


In [85]:
oe.send_command(tuvieja)

'StartedRecording'

In [9]:
oe.send_command('StopRecord')

ZMQError: Operation cannot be accomplished in current state